## 影像拼接（融合）

In [1]:
from osgeo import gdal
import numpy as np

In [2]:
path_img_1 = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\呈贡10m分辨率裁剪子区域及呈贡20m分辨率\\subset_1_of_chenggong_utm48_10m.tif"
path_img_2 = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\呈贡10m分辨率裁剪子区域及呈贡20m分辨率\\subset_2_of_chenggong_utm48_10m.tif"

In [3]:
dset_subs1 = gdal.Open(path_img_1)
dset_subs2 = gdal.Open(path_img_2)
geotrans_subs1 = dset_subs1.GetGeoTransform()
geotrans_subs2 = dset_subs2.GetGeoTransform()
print('第一幅影像的地理转换参数为：', geotrans_subs1)
print('第二幅影像的地理转换参数为：', geotrans_subs2)
x_min_subs1 = geotrans_subs1[0]
y_max_subs1 = geotrans_subs1[3]
x_min_subs2 = geotrans_subs2[0]
y_max_subs2 = geotrans_subs2[3]

第一幅影像的地理转换参数为： (268167.23598, 10.0, 0.0, 2751853.4154879693, 0.0, -10.0)
第二幅影像的地理转换参数为： (268167.23598, 10.0, 0.0, 2765453.4154879693, 0.0, -10.0)


E:\anaconda3\envs\gisenv\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:
x_max_subs1 = x_min_subs1 + dset_subs1.RasterXSize * geotrans_subs1[1]
y_min_subs1 = y_max_subs1 + dset_subs1.RasterYSize * geotrans_subs1[5]
print(x_min_subs1, x_max_subs1, y_min_subs1, y_max_subs1)

268167.23598 298137.23598 2733203.4154879693 2751853.4154879693


In [5]:
x_max_subs2 = x_min_subs2 + dset_subs2.RasterXSize * geotrans_subs2[1]
y_min_subs2 = y_max_subs2 + dset_subs2.RasterYSize * geotrans_subs2[5]
print(x_min_subs2, x_max_subs2, y_min_subs2, y_max_subs2)

268167.23598 298137.23598 2747643.4154879693 2765453.4154879693


In [6]:
x_min_mosaic = min(x_min_subs1, x_min_subs2)
x_max_mosaic = max(x_max_subs1, x_max_subs2)
y_min_mosaic = min(y_min_subs1, y_min_subs2)
y_max_mosaic = max(y_max_subs1, y_max_subs2)
print(x_min_mosaic, x_max_mosaic, y_min_mosaic, y_max_mosaic)

268167.23598 298137.23598 2733203.4154879693 2765453.4154879693


In [7]:
x_res_mosaic, y_res_mosaic = geotrans_subs1[1], geotrans_subs1[5]
print(x_res_mosaic, y_res_mosaic)

10.0 -10.0


In [8]:
x_size_mosaic = (x_max_mosaic - x_min_mosaic) / x_res_mosaic
y_size_mosaic = abs((y_max_mosaic - y_min_mosaic) / y_res_mosaic)
print(x_size_mosaic, y_size_mosaic)

2997.0 3225.0


In [9]:
x_size_mosaic, y_size_mosaic = int(x_size_mosaic), int(y_size_mosaic)
print(x_size_mosaic, y_size_mosaic)

2997 3225


In [10]:
geotrans_mosaic = [x_min_mosaic, x_res_mosaic, 0, y_max_mosaic, 0, y_res_mosaic]
print(geotrans_mosaic)

[268167.23598, 10.0, 0, 2765453.4154879693, 0, -10.0]


In [11]:
img_arry_mosaic = np.zeros(shape = (dset_subs1.RasterCount, y_size_mosaic, x_size_mosaic))
print(img_arry_mosaic.shape)

(6, 3225, 2997)


In [12]:
row_start_subs1 = int((y_max_subs1 - y_max_mosaic) / y_res_mosaic)
col_start_subs1 = int((x_min_subs1 - x_min_mosaic) / x_res_mosaic)
row_start_subs2 =int((y_max_subs2 - y_max_mosaic) / y_res_mosaic)
col_start_subs2 = int((x_min_subs2 - x_min_mosaic) / x_res_mosaic)
print(row_start_subs1, row_start_subs2, col_start_subs1, col_start_subs2)


1360 0 0 0


In [13]:
img_arry_mosaic[:, row_start_subs1 : row_start_subs1 + dset_subs1.RasterYSize, col_start_subs1 : col_start_subs1 + dset_subs1.RasterXSize] = dset_subs1.ReadAsArray()
img_arry_mosaic[:, row_start_subs2 : row_start_subs2 + dset_subs2.RasterYSize, col_start_subs2 : col_start_subs2 + dset_subs2.RasterXSize] = dset_subs2.ReadAsArray()

In [14]:
path_mosaic = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\呈贡10m分辨率裁剪子区域及呈贡20m分辨率\\subset_2_of_chenggong_utm48_10m_subs_mosaic.tif"
driver = gdal.GetDriverByName("GTiff")
dset_mosaic = driver.Create(path_mosaic, 
                            xsize = x_size_mosaic, 
                            ysize = y_size_mosaic, 
                            bands = dset_subs1.RasterCount, 
                            eType = gdal.GDT_Int16)
dset_mosaic.SetGeoTransform(geotrans_mosaic)
dset_mosaic.SetProjection(dset_subs1.GetProjection())
for i in range(dset_subs1.RasterCount):
    outband = dset_mosaic.GetRasterBand(i+1)
    outband.WriteArray(img_arry_mosaic[i])

dset_mosaic = None
dset_subs1 = None
dset_subs2 = None